In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F


#--------hyperparameters------------
batch_size = 64 # 4
block_size = 256 # 8
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4 # 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384 # 32
n_head = 6 # 4
#head_size = 6 # head_size = n_embd//n_head, 64<--384//6
n_layer = 6
dropout = 0.2 # every forward and backward pass 20% of these calculations are disabled
#---------------------------------


torch.manual_seed(1337)

with open('/content/drive/MyDrive/input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

chars = sorted(list(set(''.join(text))))
vocab_size = len(chars)
s2i = {ch:i for i, ch in enumerate(chars)}
i2s = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [s2i[c] for c in s]
decode = lambda l: ''.join([i2s[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

# data loader
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # turn off training, turn on eval
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters) # allocate space to log the losses
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # turn on training, turn off eval
    return out

class Head(nn.Module):
    """" one head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False) # (32,16) (n_emb, head_size)
        self.query = nn.Linear(n_embd, head_size, bias=False) # (32,16) (n_emb, head_size)
        self.value = nn.Linear(n_embd, head_size, bias=False) # (32,16) (n_emb, head_size)
        self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size))) # (T,T)(8,8)
        # 'tril' is not a parameter in the pytorch module. To assign it to the module we have to add it using register_buffer

        self.dropout = nn.Dropout(dropout) # we can spread out dropout layers right before the residual block connects to the highway
    def forward(self, x):
        B,T,C = x.shape # 4,8,32
        k = self.key(x) # (32,16)[4,8,32] -> B,T,head_size=16
        q = self.query(x) # B,T,head_size=16
        # compute attention scores ('affinities')
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B,T,T) @ (B,C,T) --> (B,T,T) # scaled attention
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf')) # (B,T,T)

        wei = F.softmax(wei, dim=-1) # (B,T,T)

        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) #(B,T,head_size=16)
        out = wei @ v # (B,T,head_size=16)
        return out

class MultiHeadAttention(nn.Module):
    """" multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd) # this is for residual connection, called after the concatanation operation
        self.dropout = nn.Dropout(dropout) # we can spread out dropout layers right before the residual block connects to the highway

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # residual block of self attention
        out = self.dropout(self.proj(out)) # the projection back into the residual pathway/highway
         # a simple Linear layer to transform output of residual blocks back into the highway
        return out

class FeedForward(nn.Module):
    """ a simple liner layer followed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd), # 4 * n_embd because attention paper said so and also in the next Linear layer
            # done to add extra computation power to the residual block; then scaling it back done when residual block add to the highway
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # projection layer going back into the residual highway
            nn.Dropout(dropout),# we can spread out dropout layers right before the residual block connects to the highway
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of head we'd like
        super().__init__()
        head_size = n_embd//n_head
        self.sa = MultiHeadAttention(n_head, head_size) # communication
        self.ffwd = FeedForward(n_embd)  # computation
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # residual connections-> x=gradient highway, self.sa(x)=residual blocks of self attention
        x = x + self.ffwd(self.ln2(x))
        # layer-norm is added to 'x' "before" it is being fed into self-attention and feed-forward
        # here normaliztion is done across the C-dim(32), so the B and T act as batch-dimension of normalization
        return x

class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        ##self.sa_head = Head(n_embd)
        #self.sa_head = MultiHeadAttention(4, n_embd//4) # 4 heads(communication channel of 8-dimensional self-attention)
        #self.ffwd = FeedForward(n_embd)
        # self.blocks = nn.Sequential(
        #     Block(n_embd, n_head=4),
        #     Block(n_embd, n_head=4),
        #     Block(n_embd, n_head=4),
        #     nn.LayerNorm(n_embd),
        # )
        self.blocks = nn.Sequential(*(Block(n_embd, n_head=n_head) for _ in range(n_layer)))
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        # each input token/char-int is embedded--> tok_emb
        tok_emb = self.token_embedding_table(idx) # (B 4, T 8, C=n_embd 32)
        # the position of token in that block_size of token gets embedded ->pos_emb
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C=n_embd)
        x = tok_emb + pos_emb # (B, T, C=n_embd) by broadcasting

        #x = self.sa_head(x) # apply one head of self-attention. (B,T,C)
        #x = self.ffwd(x) # (B,T,C)

        x = self.blocks(x) # (B,T,C)

        logits = self.lm_head(x) # (B, T, C=vocab_size)

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens; because of we added positional encodings, idx can never be > block_size.
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond) # get the predictions
            logits = logits[:,-1,:] # focus only one the last time step (B, C)
            probs = F.softmax(logits, dim=1) # (B,C) apply softmax to get probabilities
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) sample from the distribution
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1) append sampled index to the running sequence

        return idx



model = BigramLanguageModel()
m = model.to(device)

# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
